# GPU EXECUTION OPTIMIZATION

In [1]:
import os
#This line is to use a single GPU and not block memory on all GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

# IMPORTS

In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras import utils
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


# DATA PREPROCESSING

In [3]:
# Setting paths for training data, testing data and image dimension size 224x224
TRAIN_DIR = '/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet/train'
TEST_DIR = '/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet/test/'
IMG_SIZE = 224

# LABEL EXTRACTION

In [4]:
def create_labels(img):
    label = img.split('.')[-3]
    return label

# TRAINING DATA EXTRACTION

In [5]:
def create_train_data():
    images=[]
    training_seq_labels=[]
    
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = create_labels(img)
        training_seq_labels.append(label)
        image_path = os.path.join(TRAIN_DIR,img)
        # reading the image in graysclae
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        # resizing it to 224x224 target size
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        images.append([img])
    
    # We're shuffling in cross-validation so not needed here
    #shuffle(images)
    
    # Converting the sequence labels into integer encoded and then into one-hot encoded
    # Refer https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
    label_Encoder = LabelEncoder()
    training_onehot_labels = to_categorical(label_Encoder.fit_transform(training_seq_labels))
    
    # Saving the training images as a numpy
    training_images = np.empty((len(images), IMG_SIZE, IMG_SIZE))
    for index in tqdm(range(len(images))):
        training_images[index] = images[index][0]
    
    # Data Normalization
    training_images = ((training_images.astype('float32'))/255)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_images.npy', training_images)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_onehot_labels.npy', training_onehot_labels)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_seq_labels.npy', training_seq_labels)
    print("Training images scanned: {}".format(training_images.shape[0]))
    return training_images, training_onehot_labels, training_seq_labels

In [6]:
def extract_test_id(img):
    img_id = int(img.split('.')[0])
    return img_id

In [7]:
def create_test_data():
    images=[]
    test_id=[]
    
    for img in tqdm(os.listdir(TEST_DIR)):
        img_id = extract_test_id(img)
        test_id.append(img_id)
        image_path = os.path.join(TEST_DIR,img)
        # reading the image in grayscale
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        # resizing it to 224x224 target size
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        images.append([img])
    
    # We're shuffling in cross-validation so not needed here
    #shuffle(images)
      
    # Saving the test images as a numpy
    test_images = np.empty((len(images), IMG_SIZE, IMG_SIZE))
    for index in tqdm(range(len(images))):
        test_images[index] = images[index][0]
    
    # Data Normalization
    test_images = ((test_images.astype('float32'))/255)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_images.npy', test_images)
    np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_id.npy', test_id)
   
    print("Test images scanned: {}".format(test_images.shape[0]))
    return test_images, test_id

In [8]:
#training_images, training_onehot_labels, training_seq_labels = create_train_data()
training_images = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_images.npy')
training_onehot_labels = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_onehot_labels.npy')
training_seq_labels = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_training_seq_labels.npy')

In [9]:
#test_images, test_id = create_test_data()
test_images = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_images.npy')
test_id = np.load('/media/MyDataStor2/jaybaksh/data/ImageNet/new_test_id.npy')

# TRAINING AND VALIDATION DATA SPLIT

In [10]:
x_train, x_test, y_train, y_test = train_test_split(training_images, training_onehot_labels, test_size=0.3, random_state=40)

print("Number of training images and labels: {}, {}".format(x_train.shape, y_train.shape))
print("Number of test images and labels: {}, {}".format(x_test.shape, y_test.shape))

Number of training images and labels: (17500, 224, 224), (17500, 2)
Number of test images and labels: (7500, 224, 224), (7500, 2)


# DATA RESHAPING

In [11]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(([x_train.shape[0], 1, IMG_SIZE, IMG_SIZE]))
    x_test = x_test.reshape(x_test.shape[0], 1, IMG_SIZE, IMG_SIZE)
    input_shape = (1, IMG_SIZE, IMG_SIZE)
    
else:
    x_train = x_train.reshape(([x_train.shape[0], IMG_SIZE, IMG_SIZE, 1]))
    print("Reshaped x_train: {} and type: {}".format(x_train.shape, type(x_train)))
    print("Reshaped y_train: {} and type: {}".format(len(y_train), type(y_train)))
    
    x_test = x_test.reshape(x_test.shape[0], IMG_SIZE, IMG_SIZE, 1)
    print("Reshaped x_test: {} and type: {}".format(x_test.shape, type(x_test)))
    print("Reshaped y_test: {} and type: {}".format(len(y_test), type(y_test)))
    
    input_shape = (IMG_SIZE, IMG_SIZE, 1)

Reshaped x_train: (17500, 224, 224, 1) and type: <class 'numpy.ndarray'>
Reshaped y_train: 17500 and type: <class 'numpy.ndarray'>
Reshaped x_test: (7500, 224, 224, 1) and type: <class 'numpy.ndarray'>
Reshaped y_test: 7500 and type: <class 'numpy.ndarray'>


In [12]:
if K.image_data_format() == 'channels_first':
    test_images = test_images.reshape(([test_images.shape[0], 1, IMG_SIZE, IMG_SIZE]))
    input_shape = (1, IMG_SIZE, IMG_SIZE)
    
else:
    test_images = test_images.reshape(([test_images.shape[0], IMG_SIZE, IMG_SIZE, 1]))
    print("Reshaped test_images: {} and type: {}".format(test_images.shape, type(test_images)))
    input_shape = (IMG_SIZE, IMG_SIZE, 1)

Reshaped test_images: (12500, 224, 224, 1) and type: <class 'numpy.ndarray'>


# MODEL PARAMETERS

In [13]:
num_classes = 2

# MODEL ARCHITECTURE

In [14]:
model = Sequential()

# Layer 1
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=4,padding='valid',activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

# Layer 2
model.add(Conv2D(filters=256,kernel_size=(5,5),activation='relu',strides=1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

# Layer 3
model.add(Conv2D(filters=384,kernel_size=(3,3),activation='relu'))
model.add(Dropout(0.25))

# Layer 4
model.add(Conv2D(filters=384,kernel_size=(3,3),activation='relu'))
model.add(Dropout(0.25))

# Layer 5
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

# Pool Flatten
model.add(Flatten())

# Layer 6
model.add(Dense(4096,activation='relu'))

# Layer 7
model.add(Dense(4096,activation='relu'))

# Layer 8
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.25))

# Final Layer
model.add(Dense(num_classes,activation='softmax'))

# MODEL COMPILE

In [15]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 384)         885120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 384)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 384)         1327488   
__________

In [17]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 17500 samples, validate on 7500 samples
Epoch 1/15
17500/17500 [==============================] - 20s 1ms/step - loss: 0.6923 - acc: 0.5156 - val_loss: 0.6905 - val_acc: 0.5072
Epoch 2/15
17500/17500 [==============================] - 17s 953us/step - loss: 0.6795 - acc: 0.5733 - val_loss: 0.6587 - val_acc: 0.6116
Epoch 3/15
17500/17500 [==============================] - 17s 969us/step - loss: 0.6387 - acc: 0.6405 - val_loss: 0.6034 - val_acc: 0.6855
Epoch 4/15
17500/17500 [==============================] - 18s 1ms/step - loss: 0.5667 - acc: 0.7092 - val_loss: 0.4938 - val_acc: 0.7720
Epoch 5/15
17500/17500 [==============================] - 18s 1ms/step - loss: 0.4824 - acc: 0.7685 - val_loss: 0.5132 - val_acc: 0.7575
Epoch 6/15
17500/17500 [==============================] - 18s 1ms/step - loss: 0.4146 - acc: 0.8103 - val_loss: 0.3915 - val_acc: 0.8277
Epoch 7/15
17500/17500 [==============================] - 18s 1ms/step - loss: 0.3648 - acc: 0.8382 - val_loss: 0.3688 - val_

In [18]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.432675537761
Test accuracy: 0.848933333365


In [19]:
test_id

array([10110, 10613,  1900, ...,  6262,  9633,  4820])

In [20]:
predicted_encoded_labels = model.predict_classes(test_images,batch_size=64,verbose=1)

12500/12500 [==============================] - 3s 251us/step


In [21]:
predicted_seq_labels=[]
for label in tqdm(predicted_encoded_labels):
    if label == 0:
        predicted_seq_labels.append('cat')
    else:
        predicted_seq_labels.append('dog')

100%|██████████| 12500/12500 [00:00<00:00, 1598390.29it/s]


In [22]:
prediction = []
for index in tqdm(range(len(predicted_seq_labels))):
    prediction.append([test_id[index],predicted_seq_labels[index]])

len(prediction)

100%|██████████| 12500/12500 [00:00<00:00, 147577.84it/s]


12500

In [23]:
prediction

[[10110, 'cat'],
 [10613, 'cat'],
 [1900, 'dog'],
 [9434, 'cat'],
 [10876, 'cat'],
 [5489, 'dog'],
 [1369, 'dog'],
 [10987, 'cat'],
 [10647, 'cat'],
 [2654, 'cat'],
 [8147, 'cat'],
 [5116, 'dog'],
 [8668, 'dog'],
 [556, 'cat'],
 [10081, 'cat'],
 [253, 'dog'],
 [10245, 'dog'],
 [4971, 'dog'],
 [10082, 'dog'],
 [776, 'dog'],
 [3092, 'dog'],
 [9473, 'dog'],
 [9838, 'cat'],
 [140, 'cat'],
 [2034, 'cat'],
 [2541, 'dog'],
 [3445, 'dog'],
 [6598, 'cat'],
 [11949, 'cat'],
 [8393, 'cat'],
 [3581, 'dog'],
 [7526, 'dog'],
 [7172, 'cat'],
 [4269, 'cat'],
 [1724, 'dog'],
 [10237, 'cat'],
 [11284, 'dog'],
 [3828, 'dog'],
 [3271, 'cat'],
 [10150, 'cat'],
 [12147, 'dog'],
 [5791, 'cat'],
 [4081, 'dog'],
 [7864, 'dog'],
 [6546, 'dog'],
 [2836, 'dog'],
 [8683, 'dog'],
 [8715, 'dog'],
 [11934, 'dog'],
 [7199, 'cat'],
 [3197, 'cat'],
 [12440, 'dog'],
 [2705, 'cat'],
 [72, 'dog'],
 [11346, 'dog'],
 [3626, 'dog'],
 [9712, 'cat'],
 [2211, 'dog'],
 [4965, 'dog'],
 [4904, 'cat'],
 [2891, 'dog'],
 [11948, 'cat'

In [24]:
model.save_weights('/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet-model-weights.h5')

In [25]:
model.save('/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet-model.h5')

In [26]:
np.save('/media/MyDataStor2/jaybaksh/data/ImageNet/alexnet-prediction.npy',np.array(prediction))